In [1]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

my_llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.7,
    max_tokens=400
)

load_dotenv('./../.env')

True

## Tool Binding with LLM

### Importing External Tool - Wikipedia

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=2), 
    description="This tool can provide the real-time or even the future related information.")

### Building Custom Tool - Math Functions

In [3]:
from langchain_core.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    "Result: Addition of 2 numbers"
    return a+b

@tool
def subtract_numbers(a: int, b: int) -> int:
    "Result: Subtraction of 2 numbers"
    return a-b

@tool
def multiply_numbers(a: int, b: float) -> float:
    "Result: Multiplication of 2 numbers"
    return a*b

@tool
def divide_numbers(a: float, b: float) -> float:
    "Result: Division of 2 numbers"
    return (a/b)


toolkit = [wiki_tool, add_numbers, subtract_numbers, multiply_numbers, divide_numbers]

## Building the Agent

In [4]:
from langchain.agents import initialize_agent, AgentType
from langchain_core.prompts import ChatPromptTemplate

agent = initialize_agent(
    tools=toolkit,
    llm=my_llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# query = "My salary is 10K. Now I got an increment of 15%. What is new salary ?"
# query = "Who is the winner of IPL 2026 ?"
# query = "Who is the winner of the ICC Champions Trophy 2025 ? What is my salary after getting 15% raise on 10K ?"
# query = "Did Donald Trump win in the 2024 presedential election ?"

input_prompt = ChatPromptTemplate.from_messages([
    "system", "This is year 2025. You provide a concise response to the human query."
    "user", "Who is the latest winner of the ICC Champions Trophy ? What is my salary after getting 15% raise on 10K ?"
])

response = agent.invoke(input_prompt)

print(response['output'])

/var/folders/xl/2h_vnm9s4dl0fnn4x8_mg4q40000gn/T/ipykernel_71081/1199565432.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
<think>
Okay, let's tackle the user's questions. First, they want to know the latest winner of the ICC Champions Trophy. Since the current year is 2025, I need to check the most recent tournament. The ICC Champions Trophy was last held in 2023, so maybe there's a 2024 or 2025 event. Wait, but the user mentioned 2025, so perhaps there's a new edition. I should use the Wikipedia tool to find the latest winner.

Next, the user is asking about their salary after a 15% raise on $10K. That's a straightforward calculation. 15% of 10,000 is 1,500, so the new salary would be 11,500. I can use the add_numbers tool for this. But wait, the tools available are add, subtract, multiply, divide. So 10,000 plus 15% of 10,000. Alternatively, multiply 10,000 by 1.15. Maybe the multiply tool is better here. Let me confirm: 10,000 * 1.15 = 11,500. So using multiply_numbers with a=10000 and b=1.15.

But the user might have intended to add 15% as a separate step. Howev

## Using Playwright Browser Tool with Agent

In [5]:
%pip install --upgrade --quiet  playwright
%pip install --upgrade --quiet  lxml

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Setup the PlayWright Toolkit

In [6]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser
)

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

In [7]:
# Creating the async browser to perform the actions
async_browser = create_async_playwright_browser()

browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)

list_of_tools = browser_toolkit.get_tools()

list_of_tools


[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/par

### Setup the Agent for PlayWright

In [15]:
from langchain.agents import initialize_agent, AgentType
from langchain_core.prompts import ChatPromptTemplate

agent = initialize_agent(
    tools=list_of_tools,
    llm=my_llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "What is the address of Ramakrishna Math Pune on the https://belurmath.org/ramakrishna-math-pune/ page ?"
# query = "What are the hyperlinks on the https://belurmath.org/ramakrishna-math-pune/ page ?"

result = await agent.ainvoke(query)

result['output']



> Entering new AgentExecutor chain...
<think>
Okay, the user is asking for the address of Ramakrishna Math Pune from the page https://belurmath.org/ramakrishna-math-pune/. Let me figure out how to get that.

First, I need to navigate to the provided URL. The tool 'navigate_browser' can take me there. Once on the page, I should check if the address is listed directly. Maybe the address is in the text of the page or within a specific element like a div or paragraph.

Alternatively, I can extract all the text on the page using 'extract_text' and look for keywords like "address," "location," or "Ramakrishna Math Pune." If that's not efficient, perhaps using 'get_elements' with a CSS selector to target elements that might contain the address, such as '.address' or 'div.location', and then extract the text from those elements.

Wait, maybe the address is in a paragraph or a section with a specific class. If I can get all the text, I can search for the address. Let me first navigate to the 

'<think>\n</think>\n\nThe address of Ramakrishna Math Pune is **131-1A, Sinhagad Road, Near Dandekar Bridge, Pune, Maharashtra 411030, India**.'